In [1]:
import sys
# from utils import alignment_utils
sys.path.append('utils')

# from alignment_utils import one_file_per_process
# from alignment_utils import extract_and_combine_numbers
# from alignment_utils import extract_and_combine_numbers_in_not_start
# from alignment_utils import longest_increasing_subsequence_index
# from alignment_utils import find_answer_split_str
# from alignment_utils import find_next_question_index
# from alignment_utils import refine_answers
# from alignment_utils import match_specific_from_end
# from alignment_utils import answer_area_str_process
# from alignment_utils import remove_chinese_num_title
# from alignment_utils import align_answers_in_questions
# from alignment_utils import match_specific_from_start
# from alignment_utils import type_of_judgment
# from alignment_utils import split_question
# from alignment_utils import find_continuous_sequence
# from alignment_utils import extract_and_combine_numbers_in_not_start_by_number
# from alignment_utils import count_answer_keywords

In [43]:
import bisect
import re
import regex
import statistics
import numpy as np


GET_TOPIC_PATTERN = re.compile(
    r'^(\d+)\\*[．.]|^(\\*[（(])\d+\\*[）)]|^\\*\d+\\\.'+
    '|^\d+、|^\d+[ABCD] |^\[(\d+)\]|^\d+\（|\(\\\\text\{(\d+)\}\)'+
    '|\$\((\d+)\)\$|\$\((\\\\text\{(\d+)\})\)\$|\$\(\\\\text\{(\d+)\}\)\$'
)

EXTRACT_NUMBER_PATTERN = re.compile(
    r"^(\d+)\\*[．.]|^(\\*[（(])\d+\\*[）)]|^\\*\d+\\\.|^\d+、|^\d+[ABCD] "+
    r"|\^\[(\d+)\]|^\d+\（|^\d+【|\$\((\\text\{\d+\})\)\$"
)



REMOVE_NOISE_CHAR = re.compile(r'image\d+\.')
GET_TOPIC_PATTERN_IN_NOT_START = regex.compile(r'(\d+)\\*[．. ]|(\\*[（(])\d+\\*[）)]|\\*\d+\\\.|\d+、|^\d+[ABCD]')


def one_file_per_process(text):
    """
    接受一个可能包含多行的文本字符串，将其拆分为行，去除每一行中的某些特定字符，移除空行，然后将清理后的行重新组合成一个单一的字符串。
    
    参数:
        text (str): 可以包含多行的文本字符串。
        
    返回:
        str: 每一行都已经被清理和重新组装的文本字符串。
    """

    liens = text.splitlines()
    liens = map(lambda x: x.replace("> ", "").replace(">", "").replace("*", "").replace("\u3000", ""), liens)
    liens = filter(lambda x: x.strip() != "", liens)
    return "\n".join(liens)


def extract_and_combine_numbers(str_val):
    """
    从给定的字符串中提取数字，它会寻找字符串中开始的题号，然后从中提取出数字字符，再将它们组合在一起。
    
    参数:
        str_val (str): 输入字符串，需要从中提取数字。
        
    返回:
        int: 从输入字符串中提取出的数字。如果没有数字，就返回0。
    """
    
    match = EXTRACT_NUMBER_PATTERN.match(str_val)

    numbers = None
    if match:
        numbers = ''.join(c for c in match[0] if c.isdigit())

    return int(numbers) if numbers else None


def extract_and_combine_numbers_in_not_start(str_val):
    """
     从给定的字符串中提取数字，它会寻找字符串中任意位置开始的第一个题号，然后从中提取出数字字符，再将它们组合在一起。
    
    参数:
        str_val (str): 输入字符串，需要从中提取数字。
        
    返回:
        int: 从输入字符串中提取出的数字。如果没有数字，就返回0。
    """
    str_val = REMOVE_NOISE_CHAR.sub("", str_val)
  
    match = EXTRACT_NUMBER_PATTERN.search(str_val)

    numbers = None
    if match:
        numbers = ''.join(c for c in match.group() if c.isdigit())

    return int(numbers) if numbers else None


def extract_and_combine_numbers_in_not_start_by_number(str_val, number):
    """
     从给定的字符串中提取数字，它会寻找字符串中任意位置开始的第一个题号，然后从中提取出数字字符，再将它们组合在一起。

    参数:
        str_val (str): 输入字符串，需要从中提取数字。

    返回:
        int: 从输入字符串中提取出的数字。如果没有数字，就返回0。
    """
    str_val = REMOVE_NOISE_CHAR.sub("", str_val)
    number = str(number)
    pattern = regex.compile(r'(\d+)\\*[．.]|(\\*[（(])\d+\\*[）)]|\\*\d+\\\.|\d+、|\d+[ABCD] |\[(\d+)\]|\d+（|\d+【'.replace('\d+',number))
    match = pattern.search(str_val)

    numbers = None
    if match:
        numbers = ''.join(c for c in match.group() if c.isdigit())

    return int(numbers) if numbers else None

def longest_increasing_subsequence_index(topics):
    """
    获取一个列表中的最长递增子序列的索引。首先，它从主题列表中提取数字，然后找到最长的递增子序列，只返回开始位置在topics长度一半之前的。
    
    参数:
        topics (list): 包含一系列题目的列表，这个题目列表应该是不完全正确的
        deviation: 允许在最长的公众序列中
        
    返回:
        list: 最长递增子序列的索引列表。
    """
    # Use a list of tuples, where each tuple contains the number and its index in the original list
    nums = [(extract_and_combine_numbers(topic), i) for i, topic in enumerate(topics) if extract_and_combine_numbers(topic) is not None]
    
    topics = nums
    if not nums:
        return []

    n = len(nums)
    dp = [[i] for i in range(n)]

    for i in range(n):
        for j in range(i):
            if nums[j][0] < nums[i][0]:
                if len(dp[j]) + 1 > len(dp[i]) or (len(dp[j]) + 1 == len(dp[i]) and dp[j][-1] == dp[i][-1] - 1):
                    dp[i] = dp[j] + [i]

    # We need to filter based on the original index, not the index in the nums list
    dp = list(filter(lambda lis:nums[lis[0]][1] < len(topics)/2, dp))
    dp = list(sorted(dp, key=lambda lis:len(lis), reverse=True))
    # Return the original indices, not the indices in the nums list
    return [nums[i][1] for i in dp[0]]


ANSWER_WORDS = ["答案", "参考答案", "试题解析", "参考解答"]
ANSWER_IN_QUESTION_WORDS = ["答案", "答", "解"]

def count_answer_keywords(all_question):
    """
    在试卷中寻找答案的位置，返回一个标志值或用于分隔答题区和答案区的字符串。
    
    参数:
        all_question (list): 包含所有问题的列表。
        
    返回:
        int/str: 返回一个标志值或用于分隔答题区和答案区的字符串。如果试卷无答案返回0，答案在每道试题下方返回1
        """
    occurrence_number = 0
    for question in all_question:
        if any(answer_word in question for answer_word in ANSWER_IN_QUESTION_WORDS):
            occurrence_number += 1
            
    if occurrence_number == 0:
        return 0

    if occurrence_number > len(all_question) / 2:
        return 1
    
    return -1
    
def find_answer_split_str(question_area):

    for line in question_area.split("\n"):
        if any(answer_word in line for answer_word in ANSWER_WORDS):
            return line

    return 1


def find_next_question_index(start, lines):
    """
    从给定的开始位置搜索下一道题目的索引。
    
    参数:
        start (int): 搜索的开始位置。
        lines (list): 包含所有行的列表。
        
    返回:
        int: 下一道题目的索引位置。如果没有更多的题目，就返回行列表的长度（即：返回到行列表的末尾）。
    """

    for index in range(start + 1, len(lines)):
        match = GET_TOPIC_PATTERN.match(lines[index])

        if match:
            return index

    return len(lines)


def generate_answer_area_string(text: str, split_str):
    splited_text = text.split(split_str)

    splited_text = list(filter(lambda x:x.replace(" ","") != "", splited_text))
    return None if len(splited_text) == 1 else "\n".join(splited_text[1:])


def remove_chinese_num_title(text):
    '''
    对包含答案的文本进行处理，主要是对每一行的处理。如果一行以中文数字开始，那么就提取该行第一个阿拉伯数字以后的内容，否则就保留该行。

    Args:
    text: 一个包含答案的文本。

    Returns:
    result: 一个处理后的字符串，每一行都已经进行了处理。
    '''

    lines = text.split('\n')
    result = []
    for line in lines:
        # 检查每一行是否以中文数字开始
        if re.match('^[一二三四五六七八九十]', line):
            line = REMOVE_NOISE_CHAR.sub("", line)
            matches = GET_TOPIC_PATTERN_IN_NOT_START.findall(line)
            # 如果找到匹配，那么就提取该行第一个匹配以后的内容
            if matches:
                first_match = matches[0][0] if matches[0][0] != '' else matches[0][1]
                pos = line.find(first_match)
                result.append(line[pos:].strip())
        else:
            # 如果一行不是以中文数字开始，那么就保留该行
            result.append(line.strip())
    return "\n".join(result)


def match_specific_from_end(text, number):
    '''
    从文本的末尾开始，匹配特定的数字。

    Args:
    text: 需要匹配的文本。
    number: 需要匹配的数字。

    Returns:
    返回一个元组，包含三个元素。第一个元素是匹配的文本，第二个元素是移除匹配部分后的剩余文本，第三个元素是文本开始的数字。如果没有匹配，那么返回(None, None, None)。
    '''
    text = REMOVE_NOISE_CHAR.sub("", text)
    pattern = regex.compile(r'(\d+\\*[．. ]|\\*[（(]\d+\\*[）)]|^\\*\d+\\\.|^\d+、|\d+[ABCD])')

    matches = [m for m in pattern.finditer(text)]

    if matches:
        specific_match = None
        for match in reversed(matches):  # 反向列表，从末尾开始匹配
            match_number = int(re.search(r'\d+', match.group()).group())
            if match_number == number:
                specific_match = match
                break
        if specific_match is None:
            return None, None, None
        else:
            # 在完整文本中匹配第一个符合模式的数字
            text = REMOVE_NOISE_CHAR.sub("", text)
            start_match = GET_TOPIC_PATTERN.search(text)
            if start_match:
                match_group = start_match.group()
            else:
                start_match = GET_TOPIC_PATTERN_IN_NOT_START.search(text)
                match_group = start_match.group()

            start_number = int(re.search(r'\d+', match_group).group())  # 提取数字部分
            remaining_text = text[:specific_match.start()]  # 移除匹配部分后的剩余文本
            return text[specific_match.start():], remaining_text, start_number
    else:
        return None, None, None


def match_specific_from_start(text, number):
    '''
    从文本的开始，匹配特定的数字。

    Args:
        text (str): 需要匹配的文本。
        number (int): 需要匹配的数字。

    Returns:
        tuple: 返回一个元组，包含两个元素。
               第一个元素是匹配的文本，第二个元素是匹配到的所有文本（包括当前行）。
               如果没有匹配，返回 (None, None)。
    '''
    lines = text.splitlines()
    number = str(number)

    pattern = re.compile(r'(\d+)\\*( [．.])|(\\*[（(])\d+\\*[）)]|\\*\d+\\\.|\d+、|\d+[ABCD] |\[(\d+)\]|\d+（|\d+【'.replace('\d+',number))
    for index, line in enumerate(lines):

        match = pattern.search(line)
        if match:
            return "\n".join(lines[:index]), "\n".join(lines[index:])

    return None, None


def refine_answers(raw_answer_list):
    '''
    对一个包含未完全处理好的答案列表进行重新拆分，以使其更为完整。

    Args:
    raw_answer_list: 一个包含未完全处理好的答案的列表。

    Returns:
    refined_answers: 一个包含重新处理过的答案的列表。
    '''

    refined_answers = []
    reversed_answers = raw_answer_list[::-1]

    # 对反向答案列表中的每一个答案进行遍历
    for index, current_answer in enumerate(reversed_answers):
        # 获取下一个题目的序号，如果当前题目是列表中的最后一个，
        # 那么下一个题目的序号应该是列表中的第一个题目的序号
        if index >= len(reversed_answers) - 1:
            next_topic_number = extract_and_combine_numbers(raw_answer_list[0])
        else:
            # 否则，下一个题目的序号应该是当前题目在反向列表中的下一个题目的序号
            next_topic_number = extract_and_combine_numbers(reversed_answers[index + 1])

        # 获取当前答案对应的题目序号
        current_topic_number = extract_and_combine_numbers(current_answer)

        # 从答案的末尾开始查找，尝试匹配题目序号+1的答案
        previous_answer, remaining_text, start_number = match_specific_from_end(current_answer,
                                                                                current_topic_number + 1)

        # 如果找到了匹配的答案，那么将剩余的文本添加到所有答案的列表中
        if previous_answer:
            refined_answers.append(remaining_text)
        else:
            # 否则，直接将当前答案添加到所有答案的列表中
            refined_answers.append(current_answer)

        # 计算逻辑上的上一道题的题目序号（如现在是20题，那么"previous_topic_number"应该等于19）
        previous_topic_number = current_topic_number - 1

        # 如果下一序列的道题的题目序号等于逻辑上的上一道题的题目序号，那么继续下一次循环
        if next_topic_number == previous_topic_number:
            continue

        # 否则，将反向答案列表中的剩余部分合并成一段文本
        search_text = "/n".join(reversed_answers[:index:-1])

        # 当下一道题的题目序号小于上一道题的题目序号时，进入循环
        while next_topic_number < previous_topic_number:
            # 从文本的末尾开始查找，尝试匹配上一道题的题目序号的答案
            previous_answer, remaining_text, start_number = match_specific_from_end(search_text, previous_topic_number)

            # 如果没有找到匹配的答案
            if not previous_answer:
                refined_answers.append(search_text)
                break

            # 如果找到了匹配的答案，那么将该答案添加到所有答案的列表中
            refined_answers.append(previous_answer)
            # 计算前一道题的题目序号
            previous_topic_number -= 1
            # 更新待查找的文本
            search_text = remaining_text

    # 返回所有答案的列表
    return refined_answers


ANSWERS_KEY_WORDS_IN_QUESTIONS = ["标准答案", "试题分析", "答案", "解析", "答", "解"]


def split_text_by_keywords(text: str, keywords: list) -> dict:
    # Create the regular expression pattern for the first match from the left
    # pattern = r"(【(?:" + "|".join(re.escape(keyword) for keyword in ANSWERS_KEY_WORDS_IN_QUESTIONS) + r")】)"
    pattern = r"【[^】]*】"
    match = re.search(pattern, text)

    if not match:
        pattern = f"({'|'.join(ANSWERS_KEY_WORDS_IN_QUESTIONS)})"
        lines = text.splitlines()

        for index in range(1, len(lines)-1):
            line = lines[index]
            match = re.search(pattern, line)
            if match:
                return {
                    "question": "\n".join(lines[:index]),
                    "answer": "\n".join(lines[index:])
                }
                    
    else:
        # Get the matched text
        matched_text = match.group(0)

        # Split the text into two parts at the matched text
        first_part, second_part = text.split(matched_text, 1)

        # Return the result as a dictionary
        return {
            "question": first_part.strip(),
            "answer": matched_text + second_part.strip()
        }
    


def align_answers_in_questions(questions: list, keywords: list = ANSWERS_KEY_WORDS_IN_QUESTIONS) -> list:
    answers_with_questions = []
    for question in questions:
        answer_with_question = split_text_by_keywords(question, keywords)
        if answer_with_question:
            answers_with_questions.append(answer_with_question)

    return answers_with_questions


def type_of_judgment(text):
    """判断文本类型。

    该函数接受一个文本块，并根据问题编号列表中数字 "1" 的出现情况来确定判断类型。函数假设文本以 "一、"、"二、" 等形式进行编号。

    参数：
        text (str)：包含问题及其编号的输入文本。

    返回：
        bool：如果数字 "1" 出现次数超过问题数量的一半，则返回 True，否则返回 False。
    """
    
    # 将文本拆分成行
    lines = text.splitlines()
    print(lines)
    # 初始化一个空列表用于存储问题编号
    question_number_list = []
    
    # 定义一个正则表达式模式，用于匹配问题编号如 "一、"、"二、" 等
    pattern = r'^[一二三四五六七八九十][、．.]'
    
    # 遍历文本中的每一行
    for i in range(len(lines)):
        line = lines[i]
        
        # 检查当前行是否匹配问题编号的模式
        match = re.match(pattern, line)
        
        if match:
            # 如果匹配成功，从下一行中提取并组合数字，并将其添加到列表中
            if i + 1 < len(lines):
                next_line = lines[i+1]
                question_number_list.append(extract_and_combine_numbers(next_line))
    
    # 统计问题编号列表中数字 "1" 的出现次数
    count_of_ones = question_number_list.count(1)

    # 如果数字 "1" 出现次数超过问题数量的一半，则返回 True，否则返回 False
    return count_of_ones >= len(question_number_list) / 2


def split_question(question, topic_number=0, target_topic_number=0):
    """
    将问题拆分成子问题的函数。

    该函数将给定问题拆分为多个子问题，并返回一个子问题的列表。每个子问题以特定编号开始，编号从 `topic_number` 开始。
    搜索将从 `target_topic_number` 结束。如果未指定 `topic_number` 和 `target_topic_number`，则默认值为0。

    Args:
        question (str): 要拆分的问题文本。
        topic_number (int, optional): 子问题的起始编号。默认值为0。
        target_topic_number (int, optional): 拆分的终止编号。默认值为0。

    Returns:
        list: 包含拆分后的子问题的列表。
    """
    # 如果未指定 topic_number，则从问题文本中提取并组合数字作为起始编号
    topic_number = topic_number if topic_number else extract_and_combine_numbers_in_not_start(question)
    
    # 如果未指定 target_topic_number，则默认情况下进行递归搜索直到结束
    recursion = False if target_topic_number else True
    target_topic_number = target_topic_number if target_topic_number else topic_number + 1

    # 初始化用于存储拆分后子问题的列表
    split_question_list = []
    
    # 从问题文本和起始编号开始搜索子问题
    search_question = question
    search_number = topic_number + 1
    
    # 循环搜索子问题，直到达到目标编号或无法找到更多子问题
    while True:
        # 使用函数 match_specific_from_start 在搜索文本中匹配特定编号的子问题
        current_question, next_question = match_specific_from_start(search_question, search_number)
        
        # 如果找到子问题，将其添加到拆分后的子问题列表中，并更新搜索文本为下一个问题的起始位置
        if current_question is not None:
            split_question_list.append(current_question)
            search_question = next_question
        else:
            break
            
        # 如果已经达到目标编号，而且不需要进行递归搜索，停止拆分
        if target_topic_number - search_number <= 0:
            if not recursion:
                break

        # 增加搜索编号，以搜索下一个子问题
        search_number = search_number + 1
        
    # 将最后一个子问题添加到列表中
    split_question_list.append(search_question)
   
    # 返回拆分后的子问题列表
    return split_question_list


def find_continuous_sequence(all_question):
    """
    查找连续编号问题序列的函数。

    该函数处理给定的问题列表 `all_question`，并查找其中连续编号的问题序列。
    连续编号的问题序列指的是问题编号相邻且连续的问题组成的序列。

    Args:
        all_question (list): 包含问题文本的列表。

    Returns:
        list: 包含连续编号问题序列的列表。
    """
    # 初始化一个新的问题列表，用于存储连续编号问题序列
    new_all_question = []
    
    # 遍历所有问题
    for index, question in enumerate(all_question):
        # 检查是否到达问题列表的末尾
        if index + 1 == len(all_question):
            # 如果到达末尾，则将当前问题拆分后添加到新的问题列表中，然后跳出循环
            new_all_question.append(question)
            break
            
        # 提取当前问题的编号
        topic_number = extract_and_combine_numbers(question)
        
        # 提取下一个问题的编号
        next_topic_number = extract_and_combine_numbers(all_question[index+1])
        
        # 如果当前问题编号加1等于下一个问题编号，则它们是连续的
        if topic_number + 1 == next_topic_number:
            # 将当前问题添加到新的问题列表中
            new_all_question.append(question)
            continue
        
        # 如果当前问题编号和下一个问题编号不连续，则需要拆分当前问题，并将拆分后的子问题添加到新的问题列表中
        new_all_question += split_question(question, topic_number, next_topic_number-1)
        
    # 返回包含连续编号问题序列的新问题列表
    return new_all_question


# 读取题目

In [44]:
file_path = r"C:\Users\Zhipeng\Desktop\test\input\2009年全国高考贵州省文综地理试题及答案.md"
import re
regex_pattern = r"!\[\]\(media/(.+?)\)"
with open(file_path, 'r', encoding='utf-8') as file:
    md_content = file.read()
    modified_text = re.sub(regex_pattern, r"![](media/\1)\n", md_content)

modified_text.splitlines()

['2009年普通高等学校招生统一考试试题卷',
 '',
 '文科综合能力测试',
 '',
 '**第Ι卷（选择题）**',
 '',
 '本卷共35小题，每小题4分，共140分。在每题给出的四个选项中，只有一项是符合题目要求的。',
 '',
 '近期研制出利用玉米叶片加工、编织购物袋的技术，这种购物袋易分解且物美价廉。据此完成1\\~3题。',
 '',
 '1.该种购物袋的生产厂应该近',
 '',
 'A．原料产地 B.销售产地',
 '',
 'C.能原产地 D.研发基地',
 '',
 '2.以该种购物袋替代目前广泛使用的同类产品，对环境保护的直接作用是',
 '',
 'A.减轻大气污染 B.减轻"白色污染"',
 '',
 'C.促进生物多样性 D.减轻酸雨危害',
 '',
 '3.该种购物袋的生产原料在我国的最大产地是',
 '',
 'A.河南省 B.山东省',
 '',
 'C.四川省 D.吉林省',
 '',
 '图1显示某国1951---2000年每10年的人口变化![](media/image1.jpeg)',
 '读图1完成4～5题',
 '',
 '[![](media/image2.jpeg)',
 '](http://www.ks5u.com/)',
 '',
 '4.1951------2000年，该国',
 '',
 'A.是人口净迁入国 B.人口的迁入是维持人口增长的主要方式',
 '',
 'C.人口的出生率一直高于死亡率 D.人口增长了15.1%',
 '',
 'w.w.w.k.s.5.u.c.o.m',
 '',
 '5.据图示资料推测，该国',
 '',
 'A.自20世纪70年代，人口再生产进入现代型',
 '',
 'B.可能位于欧洲',
 '',
 'C.1990年人口达到900万',
 '',
 'D.20世纪末青壮年人口比重偏大',
 '',
 '图2中。H地恰与某高压天气系统中心吻合，该天气系统以每天约200千米的速度东移。据此完成6～8题。',
 '',
 '[![](media/image3.jpeg)',
 '](http://www.ks5u.com/)',
 '',
 '6.H地与甲聚落的相对高差约为',
 '',
 'A. 800米 B.1500米',
 '',
 'C. 1800米 

# 模板匹配

# detect

In [45]:
text=modified_text
print(f"【StandardExamParser开始】")


【StandardExamParser开始】


# 寻找题目

In [46]:
#去除大写数字，如三、解答题这种标题行。
remove_title_text= remove_chinese_num_title(text)

lines =remove_title_text.splitlines()

# 定义不准确的题目列表    
inaccuracy_question = []

# 从0的位置寻找第一道题
index = find_next_question_index(0, lines)

while index < len(lines):
    # 寻找下一个题目的index
    next_index = find_next_question_index(index, lines)

    inaccuracy_question.append("\n".join(lines[index: next_index]))
    index = next_index

#     print([(extract_and_combine_numbers(topic), i) for i, topic in enumerate(inaccuracy_question) if extract_and_combine_numbers(topic) is not None])
# 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)


# 定义准确的题目列表    
all_question = []
# index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
for index, question_index in enumerate(all_question_indexs):
    if index == len(all_question_indexs) -1 :
        all_question.append("\n".join(inaccuracy_question[question_index:]))
        break
#这一步是将题目分隔开，试卷末尾的答案被暂时分在最后一题里
    all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
#     for i in all_question:
#         print(i)
#         print("==========")


In [47]:
all_question = find_continuous_sequence(all_question)

#判断为standard模式：找到"答案", "参考答案", "试题解析", "参考解答"所在行，并分割。
answer_split_str = find_answer_split_str(all_question[-1])

if isinstance(answer_split_str, str):
    all_answer_area=all_question[-1].split(answer_split_str)[1]
    all_question[-1] = all_question[-1].split(answer_split_str)[0]

answer_keyword_num=count_answer_keywords(all_question)

#判断是否出现"答案", "答", "解"，没有出现则找不到答案。
if answer_keyword_num==0 or answer_keyword_num==-1:
    pass

#判断是否为annotated模式:大多数题都出现"答案", "答", "解"
if answer_keyword_num==1:
    pass



In [51]:
lines = all_answer_area.splitlines()

In [52]:


inaccuracy_answers = []

index = find_next_question_index(0, lines)
while index < len(lines):
    next_index = find_next_question_index(index, lines)

    inaccuracy_answers.append("\n".join(lines[index: next_index]))
    index = next_index

#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()

#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)



In [53]:
processed_inaccuracy_answers = []
for index, answer_index in enumerate(inaccuracy_answer_indexes):
    if index == len(inaccuracy_answer_indexes) -1 :
        processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
        break
    processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))


In [54]:
refine_answer=refine_answers(processed_inaccuracy_answers)[::-1]

In [55]:
for answer in refine_answer:
    print(answer[:10])

1.A 
2.B 
3.D 
4.A 
5.B

6.B 
7.D 
8.C 
9.B 
10.C

11.A

第Ⅱ卷共
1.A 2.B 3.
36.（36分）


1.A 2.B 3.
39.（4）交通区位


In [28]:

try:
    question_count = len(all_question)
    answer_count = len(inaccuracy_answers)
    print(f"question_list:'{question_count}'")
    print(f"answer_list:'{answer_count}'")
    is_match = question_count == answer_count
except:
    print(f"【抽取题干答案报错】")
    


if is_match:
    print(f"【题干答案已匹配】")
    print(f"【StandardExamParser匹配成功】")
else:
    print(f"【题干答案不匹配】")
    print(f"【StandardExamParser匹配失败】")

question_list:'23'
answer_list:'23'
【题干答案已匹配】
【StandardExamParser匹配成功】


# 创建测试样本

['绝密★启用前',
 '',
 '2008年普通高等学校招生全国统一考试（江西卷）',
 '',
 '**文科数学**',
 '',
 '本试卷分第Ⅰ卷（选择题）和第Ⅱ卷（非选择题）两部分，第Ⅰ卷1至2页，第Ⅱ卷3至4页，共150分。',
 '',
 '**第Ⅰ卷**',
 '',
 '**考生注意：**',
 '',
 '1.  答题前，考生务必将自己的准考证号、姓名填写在答题卡上，考生要认真核对答题卡上粘贴的条形码的"准考证号、姓名、考试科目"与考生本人准考证号、姓名是否一致。',
 '',
 '2.  第I卷每小题选出答案后，用2B铅笔把答题卡上对应题目的答案标号涂黑，如需改动，用橡皮擦干净后，再选涂其他答案标号。第Ⅱ卷用黑色墨水签字笔在答题卡上作答。若在试题卷上作答，答案无效。',
 '',
 '3.  考试结束，监考员将试题卷、答题卡一并收回。',
 '',
 '参考公式',
 '',
 '如果事件$A,B$互斥，那么 球的表面积公式',
 '',
 '$P(A + B) = P(A) + P(B)$ $S = 4\\pi R^{2}$',
 '',
 '如果事件$A,B$，相互独立，那么 其中$R$表示球的半径',
 '',
 '$P(A \\cdot B) = P(A) \\cdot P(B)$ 球的体积公式',
 '',
 '如果事件$A$在一次试验中发生的概率是$p$，那么 $V = \\frac{4}{3}\\pi R^{3}$',
 '',
 '$n$次独立重复试验中恰好发生$k$次的概率 其中$R$表示球的半径',
 '',
 '$P_{n}(k) = C_{n}^{k}p^{k}(1 - p)^{n - k}$',
 '',
 '**一．选择题：本大题共12小题，每小题5分，共60分。在每小题给出的四个选项中，只有一项是符合题目要求的．**',
 '',
 '1．"$|x| = |y|$"是"$x = y$"的',
 '',
 'A．充分不必要条件 B．必要不充分条件 C．充要条件 D．既不充分也不必要条件',
 '',
 '2．定义集合运算:$A*B = \\left\\{ \\left. \\ z \\right|z = xy,x \\in A,y \\in B \\right\\}$.设$A = \\left\\{ 1,2 \\ri

In [28]:
if split_str == 1:
    answers_with_questions = align_answers_in_questions(all_question)
#     print(check_sequence_is_full(answers_with_questions))
    for row in align_answers_in_questions(all_question):
        print()
        print(row["question"][:3])
        print("-------------------------------------------------------")
        print(f"{row['answer']}")
        print()
        print("============================================================")

# 寻找答案（试卷存在答题区与答案区）

In [33]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print("=================")

1．B.因$|x| = |y| \nRightarrow x = y$但$x = y$
![](media/image28.wmf)
$|x| = |y|$。

2．$D$.因$A*B = \{ 0,2,4\}$，

3．B.
因为$f(x)$的定义域为\[0，2\]，所以对$g(x)$，$0 \leq 2x \leq 2$但$x \neq 1$故$x \in \lbrack 0,1)$。

4．$C$ 函数$f(x) = \log_{4}x$为增函数

5．$A$
$a_{2} = a_{1} + \ln(1 + \frac{1}{1})$，$a_{3} = a_{2} + \ln(1 + \frac{1}{2})$，...，$a_{n} = a_{n - 1} + \ln(1 + \frac{1}{n - 1})$

$$\Rightarrow a_{n} = a_{1} + \ln(\frac{2}{1})(\frac{3}{2})(\frac{4}{3})\cdots(\frac{n}{n - 1}) = 2 + \ln n$$

6．$A$
$f( - x) = \frac{\sin( - x)}{\sin( - x) + 2\sin\frac{- x}{2}} = f(x)$
$f(4\pi + x) = f(x) \neq f(2\pi + x)$

7\.
$C$.由题知,垂足的轨迹为以焦距为直径的圆,则$c < b \Rightarrow c^{2} < b^{2} = a^{2} - c^{2} \Rightarrow e^{2} < \frac{1}{2}$

又$e \in (0,1)$,所以$e \in (0,\frac{1}{2})$

8\. $D$
$(1 + x)^{10}(1 + \frac{1}{x})^{10} = \frac{(1 + x)^{20}}{x^{10}}$

9\. $C$.

10．$D$..函数$y = \tan x + \sin x - \left| \tan x - \sin x \right| = \left\{ \begin{matrix}
2\tan x,\mspace{6mu}\mspace{6mu} 当\tan x < \sin x时 \\
2\sin x,\mspace{6mu}\mspac

# 对齐

In [34]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    if not answer_str:
        return [{"question":question, "answer":None}for question in all_question]
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))
    
    #  倒叙是因为，有的文件出现多个相同的题号，我们确保我们拿到的是准确的
    questions_map = {extract_and_combine_numbers_in_not_start(question):question for question in reversed(all_question)} 
    answer_map = {extract_and_combine_numbers_in_not_start(answer):answer for answer in reversed(all_answer)}

    for sequence_number in questions_map:
        questions_with_answer.append({
            "question":questions_map.get(sequence_number),
            "answer":answer_map.get(sequence_number, None)
        })
        
    return list(reversed(questions_with_answer))

In [35]:
def check_sequence_is_full(questions_with_answer):
    for index, question_map in enumerate(questions_with_answer):
        if index == len(questions_with_answer)-1:
            break
            
        current_number = extract_and_combine_numbers_in_not_start(question_map["question"])
        next_number = extract_and_combine_numbers_in_not_start(questions_with_answer[index+1]["question"])
        
        if not current_number or not next_number:
            return False
        
        if current_number + 1 != next_number:
            return False
      
    return True

In [36]:
for row in alignment_answer(all_question, answer_str):
    print()
#     print(row["question"][:2])
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


1．"$|x| = |y|$"是"$x = y$"的

A．充分不必要条件 B．必要不充分条件 C．充要条件 D．既不充分也不必要条件

-------------------------------------------------------
1．B.因$|x| = |y| \nRightarrow x = y$但$x = y$
![](media/image28.wmf)
$|x| = |y|$。



2．定义集合运算:$A*B = \left\{ \left. \ z \right|z = xy,x \in A,y \in B \right\}$.设$A = \left\{ 1,2 \right\}$,$B = \left\{ 0,2 \right\}$,则集合$A*B$
的所有元素之和为

A．0 B．2 C．3 D．6

-------------------------------------------------------
2．$D$.因$A*B = \{ 0,2,4\}$，



3．若函数$y = f(x)$的定义域是$\lbrack 0,2\rbrack$，则函数$g(x) = \frac{f(2x)}{x - 1}$的定义域是

A．$\lbrack 0,1\rbrack$ B．$\lbrack 0,1)$ C．
$\lbrack 0,1) \cup (1,4\rbrack$ D．$(0,1)$

-------------------------------------------------------
3．B.
因为$f(x)$的定义域为\[0，2\]，所以对$g(x)$，$0 \leq 2x \leq 2$但$x \neq 1$故$x \in \lbrack 0,1)$。



4．若$0 < x < y < 1$，则

A．$3^{y} < 3^{x}$ B．$\log_{x}3 < \log_{y}3$ C．$\log_{4}x < \log_{4}y$
D．$(\frac{1}{4})^{x} < (\frac{1}{4})^{y}$

-------------------------------------------------------
4．$C$ 函数$f(x) = \log_{4}x$为增函数



5．

# 生产测试

In [ ]:
import csv
from tqdm import tqdm

csv_path = "./questions_with_answers.csv"
loss_answer_file = set()
loss_question_file = set()
error_file = set()
check_error_file = set()

with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['file_path', 'question', 'answer', 'is_full', 'is_error']  # 列名
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for examination_paper in tqdm(examination_paper_list):
#         try:
            file_path = examination_paper["file_path"]
            text = examination_paper["text"]

            all_question, split_str = get_all_question(text)

            # 判断试卷你是否有答案 -1 识别不到 0 无答案 
            if split_str in [-1, 0]: 
                loss_answer_file.add(file_path)
                continue

            # 无法找到题目
            if not all_question:
                loss_question_file.add(file_path)
                continue

            # 答案在每道题的下方split_str == 1，如果答案在卷子下方与题目隔离则split_str是返回隔离答案和题目那一行的str字符串
            if split_str == 1:
                questions_with_answers = align_answers_in_questions(all_question)
            else:
                answer_str = generate_answer_area_string(text, split_str)
                questions_with_answers = alignment_answer(all_question, answer_str)

            #  如果题目与答案都无法匹配
            if questions_with_answers is []:
                error_file.add(file_path)
                continue
            
            is_full = check_sequence_is_full(questions_with_answers)
           
            if not is_full:
                check_error_file.add(file_path)

            for question_with_answer in questions_with_answers:

                if question_with_answer is None:
                    question_with_answer = {"question":None,"answer":None}

                question_with_answer.update({"is_full":is_full,"file_path":file_path,"is_error":False})

                if question_with_answer is None:
                    question_with_answer.update({"is_error":True})

                if len(question_with_answer) == 5:
                        writer.writerow(question_with_answer)
#                 else:
#                     print(question_with_answer)
#         except:
#             print(file_path)

  6%|████▋                                                                        | 116/1886 [00:00<00:01, 1137.75it/s]

In [ ]:
len(loss_answer_file | loss_question_file | error_file | check_error_file) / len(examination_paper_list)

In [ ]:
len(check_error_file)

In [20]:
check_error_file

{'filter_examination_pater_markdown\\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md',
 'filter_examination_pater_markdown\\1962年试题.md',
 'filter_examination_pater_markdown\\1963年试题.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试(安徽卷)数学文科详细解析（包括选择题填空题）.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试山东卷（数学文）.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试辽宁卷（数学文）.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（四川卷）文综试卷参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（宁夏卷）数学（理科）参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（江西卷）数学（文科）参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（江西卷）数学（理科）参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（湖北卷）数学（理工农医类）参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（福建卷）数学（文科）试卷参考答案.md',
 'filter_examination_pater_markdown\\2007年普通高等学校招生全国统一考试（重庆卷）文综试卷参考答案.md',
 'filter_examination_pater_markdown\\2007年考研数学三真题及解析.md',
 'fil

In [21]:
get_all_question(test_text)

(['1．7.8＋7.8＋7.8＋7.8＋7.8=( )×( )。',
  '2．将6.58的小数点向右移动一位，这个数就扩大到原来的( )倍，结果是( )。',
  '3．两个因数的积是0.56，如果把这两个因数都扩大为原来的10倍，那么积是( )。',
  '4．0.37×12.94的积是( )位小数，0.35×2.04的积化简后是( )位小数。',
  '5．根据规律填出横线上的数。\n1.7，3.4，6.8\\_\\_\\_\\_\\_\\_\\_，\\_\\_\\_\\_\\_\\_\\_。来源：www.bcjy123.com/tiku/',
  '6．2.5缩小到原来的是( )，再扩大1000倍是( )。',
  '7．在下面的括号里填上"＞"、"＜"或"="。\n653×0.9( )653 1×0.99( )1\n5.37×1.1( )5.37 0.54×1( )5.4×0.1',
  '8．去掉3.045的小数点，得到的数是( )，原数就扩大了( )倍。',
  '9．25×10.2=25×10＋( )×25，运用了( )。',
  '10．0.375这个数中的数字"7"在( )位上，如果把这个数扩大1000倍，"7"在( )上。\n二、辨一辨。(对的打"√"，错的打"×")(5分)\n1．在计算小数乘法时，和计算小数加减法一样，把小数点对齐再计算。 ( )\n2．6.5×0.24的积化简后是两位小数。 ( )\n3．整数乘法的运算律不适用小数乘法。 ( )\n4．一个数去掉小数点后扩大了10倍，这个数一定是一位小数。 ( )\n5．两个小数相乘，积的小数位数等于两个乘数的小数位数之和。 ( )\n三、选一选。(把正确答案的序号填在括号里)(5分)\n1．把0.07的小数点向左移动一位，又向右移动三位，得( )。\nA．0.7 B.7 C.70\n2．下列算式中与3.84×2.65的积相等的是( )。\nA．3.84×26.5 B．38.4×0.265 C．0.384×0.265\n3．用简便方法计算25.5×99＋25.5=25.5×(99＋1)时运用了乘法( )。\nA．交换律 B．分配律 C．结合律\n4．235×14=3290，那么2.35×1.4的结果是( )。\nA．3.29 B．0.329 C．32.9\n5．买10千克大米需付28.4

In [24]:
def check_error_line(check_error_file):
    error_line = []
    for file_path in check_error_file:
        test_text = list(filter(lambda row:file_path in row["file_path"], examination_paper_list))[0]["text"]
        all_question, split_answer_str = get_all_question(test_text)
        
        for index, question in enumerate(all_question):
            if index+1 == len(all_question):
                break
            if extract_and_combine_numbers_in_not_start(question) is None:
                print(question)
                continue
            number = extract_and_combine_numbers_in_not_start(question)
            if number + 1 != extract_and_combine_numbers_in_not_start_by_number(all_question[index + 1], number + 1):
                error_line.append({"file_path":file_path, "error_question": question, "error_next_questino":all_question[index + 1]})
    return error_line
error_line = check_error_line(check_error_file)












∴S~扇形OAC~＝![](./data/image/media/image125.png)＝![](./data/image/media/image126.png)，

∵CH＝![](./data/image/media/image127.png)＝![](./data/image/media/image128.png)，
∴S~△OHC~＝![](./data/image/media/image124.png)×1×![](./data/image/media/image128.png)＝![](./data/image/media/image129.png)，

∴r＝![](./data/image/media/image131.png)，则OH＝![](./data/image/media/image132.png)，


在Rt△ABH中，AH＝3，BH＝![](./data/image/media/image133.png)+1＝![](./data/image/media/image134.png)，则AB＝![](./data/image/media/image135.png)，
在Rt△ACH中，AH＝3，CH＝NH＝1，得AC＝![](./data/image/media/image136.png)，
在△BMN和△BCA中，
∠B＝∠B，∠BMN＝∠BCA，
∴△BMN∽△BCA，




∴![](./data/image/media/image137.png)＝![](./data/image/media/image138.png)即![](./data/image/media/image139.png)＝![](./data/image/media/image140.png)＝![](./data/image/media/image141.png)，
∴MN＝![](./data/image/media/image142.png)，

∴OH＝![](./data/image/media/image143.png)，MN＝![](./data/image/media/image144.png)．













































A. 1 B. ![](./

In [25]:
for line in error_line:
    print("error_file_path==>"+line["file_path"])
    print("-----------------------------------------")
    print("error_question==>\n"+line["error_question"])
    print("-----------------------------------------")
    print("error_next_questino==>\n"+line["error_next_questino"])
    print()
    print()
    print("=========================================")
    print()
    print()
    

error_file_path==>filter_examination_pater_markdown\理数8答案.md
-----------------------------------------
error_question==>
12、\[解析\] 不妨设P~1~(x~1~，y~1~)，P~2~(x~2~，y~2~)，其中0\<x~1~\<1\<x~2~.
由l~1~，l~2~分别是点P~1~，P~2~处的切线，且f′(x)＝
得l~1~的斜率k~1~＝－，l~2~的斜率k~2~＝.
又l~1~与l~2~垂直，且0\<x~1~\<x~2~，所以k~1~·k~2~＝－·＝－1⇒x~1~·x~2~＝1，
l~1~：y＝－(x－x~1~)－ln x~1~①，l~2~：y＝(x－x~2~)＋ln x~2~②，
则点A的坐标为(0，1－ln x~1~)，点B的坐标为(0，－1＋ln x~2~)，
由此可得\|AB\|＝2－ln x~1~－ln x~2~＝2－ln(x~1~·x~2~)＝2.
联立①②两式可解得交点P的横坐标x~P~＝＝，
所以S~△PAB~＝\|AB\|·\|x~P~\|＝×2×＝≤1，当且仅当x~1~＝，即x~1~＝1时，等号成立．
而0\<x~1~\<1，所以0\<S~△PAB~\<1，故选A.
二、填空 3；.；；(2，8)
-----------------------------------------
error_next_questino==>
16、\[解析\] 由已知得a＝1，b＝，c＝2.当∠F~1~F~2~P＝时，\|PF~2~\|＝3，\|PF~1~\|＝\|PF~2~\|＋2a＝5，则\|PF~1~\|＋\|PF~2~\|＝8；当∠F~1~PF~2~＝时，设\|PF~1~\|＝m，\|PF~2~\|＝n，则而(m－n)^2^＝4＝m^2^＋n^2^－2mn＝16－2mn，所以mn＝6，则(m＋n)^2^＝m^2^＋n^2^＋2mn＝28，则m＋n＝2.又△F~1~PF~2~为锐角三角形，故\|PF~1~\|＋\|PF~2~\|的取值范围是(2，8)．
三.解答题




error_file_path==>filter_examination_pater_markdown\2008年高考理综化学部分宁夏卷.md
-----

# 检验

In [26]:
import pandas as pd

df = pd.read_csv(csv_path)

# 计算每个file_path的总行数和非空answer的行数
file_path_counts = df['file_path'].value_counts()
file_path_answer_counts = df[df['answer'].notnull()]['file_path'].value_counts()

# 对齐这两个Series
file_path_counts, file_path_answer_counts = file_path_counts.align(file_path_answer_counts, fill_value=0)

# 找出那些所有相关行的answer都不为空的file_path
file_paths_with_all_answers = file_path_counts[file_path_counts == file_path_answer_counts]

# 计算这些file_path的数量
count_file_path_with_all_answers = len(file_paths_with_all_answers)

# 统计所有file_path的数量
count_all_file_path = df['file_path'].nunique()

count_file_path_with_all_answers / count_all_file_path

0.679745889387145

In [27]:
import pandas as pd

df = pd.read_csv(csv_path)

# 计算每个file_path的总行数和非空answer的行数
file_path_counts = df['file_path'].value_counts()
file_path_answer_counts = df[df['answer'].notnull()]['file_path'].value_counts()

# 对齐这两个Series
file_path_counts, file_path_answer_counts = file_path_counts.align(file_path_answer_counts, fill_value=0)

# 找出那些所有相关行的answer都不为空的file_path
file_paths_with_all_answers = file_path_counts[file_path_counts == file_path_answer_counts]

# 计算这些file_path的数量
count_file_path_with_all_answers = len(file_paths_with_all_answers)

# 统计所有file_path的数量
count_all_file_path = df['file_path'].nunique()

count_file_path_with_all_answers / count_all_file_path

0.679745889387145

In [28]:
df[df['answer'].isnull()]['file_path'].unique()

array(['answer_markdown\\2003年春季高考（北京）数学（理）试题.md',
       'answer_markdown\\2003数学（文）.md', 'answer_markdown\\2004-7数学（文）.md',
       'answer_markdown\\2004-数学（文）3.md',
       'answer_markdown\\2004-数学（文）8.md',
       'answer_markdown\\2004-数学（文）老课程.md',
       'answer_markdown\\2004-数学（理）.md',
       'answer_markdown\\2004-数学（理）7.md',
       'answer_markdown\\2004-数学（理）78.md',
       'answer_markdown\\2004年上海市普通高等学校春季招生考.md',
       'answer_markdown\\2004数学辽宁.md', 'answer_markdown\\2004数学（文）3.md',
       'answer_markdown\\2004数学（理）天津.md',
       'answer_markdown\\2004数学（理）浙江.md', 'answer_markdown\\2004春季题.md',
       'answer_markdown\\2005年高考数学（广东卷）试题及答案.md',
       'answer_markdown\\2005年高考文科数学全国卷Ⅱ试题及答案（黑龙江、吉林、广西、内蒙古、新疆等地区用）.md',
       'answer_markdown\\2005年高考文科数学全国卷Ⅲ试题及答案（川、陕、云、甘）.md',
       'answer_markdown\\2005年高考文科数学（山东卷）试题及答案.md',
       'answer_markdown\\2005年高考文科数学（江西卷）试题及答案.md',
       'answer_markdown\\2005年高考文科数学（湖南卷）试题及答案.md',
       'answer_markdown\\2005年高考理科数学全国卷Ⅱ试题及

In [29]:
for _, row in df_include_answer.iloc[10000:12050].iterrows():
    print(row['question'])
    print("************************************************************")
    print(row['answer'])
    print("\n=======================================================================\n")

NameError: name 'df_include_answer' is not defined

In [30]:
df[df['answer'].notnull()]

,file_path,question,answer,is_full,is_error
0,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,1\. 已知集合，，则 [ ]{.underline},1\.,True,False
1,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,2\. 已知，且满足，求 [ ]{.underline},2\. ，,True,False
2,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,3\. 已知向量，，则与的夹角为 [ ]{.underline},3\. ，,True,False
3,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,4\. 已知二项式，则展开式中含项的系数为 [ ]{.underline},4\. ，的系数为,True,False
4,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,5\. 已知、满足，求的最小值为 [ ]{.underline},5\. ，线性规划作图，后求出边界点代入求最值，当，时，,True,False
...,...,...,...,...,...
100237,filter_examination_pater_markdown\（精校版）2020年浙江...,22．（本题满分15分）\n已知，函数，其中e=2.71828...是自然对数的底数．\n（...,22．本题主要考查函数的单调性、零点，导数的运算及其应用，同时考查数学抽象、逻辑推理与数学运...,True,False
100250,filter_examination_pater_markdown\（精校版）2020年浙江...,13.小李收集了手机网民互联网应用的用户规模数据，并用Eel软件进行数据处理，如图a所示。\...,13.（1）=（D4-\$C4）/\$C4\100 （2）C8：F8 （3）①③或③①,True,False
100251,filter_examination_pater_markdown\（精校版）2020年浙江...,"14.小李使用Flash软件创作""花中君子""多媒体作品。请回答下列问题：\n（1）作品的模块...","14.（1）BC （2）矢量图 不会 （3）ABE\n（4）在""文字""图层第31帧处执行""插...",True,False
100252,filter_examination_pater_markdown\（精校版）2020年浙江...,15.某校为学生期末考试分配考场，并编制准考证号。每个班级有班号，每位学生有班内序号，班内序...,15.（1）A （2）①f=True ②m ③num（1）=0或等价表达式\n（3）room...,True,False
